In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import glob
from read_csv_gz import read_csv_gz
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
!pip install sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration

# model_name = "hossboll/clinical-t5"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
input_text = ["patient is suffering from chest pain", "patient is suffering from abdominal pain"]
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, padding=True, truncation=True)
with torch.no_grad():
    outputs = model.encoder(**inputs)
feature_vector = outputs.last_hidden_state.mean(dim=1)
print(feature_vector.shape)
# summary = tokenizer.decode(feature_vector, skip_special_tokens=True)
# print(summary)


torch.Size([2, 768])


In [4]:
print(feature_vector)

tensor([[-0.1681,  0.0800, -0.0874,  ..., -0.0414, -0.1860, -0.2086],
        [-0.1478,  0.0929, -0.0580,  ..., -0.0216, -0.1594, -0.1226]])


In [5]:
summary_ids = model.generate(inputs.input_ids, num_beams=4, max_length=150, early_stopping=True)
summary = tokenizer.decode(summary_ids, skip_special_tokens=True)
print(summary)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'